# Metadata

```{yaml}
Course:   DS 5001 
Module:   02 Lab
Topic:    Text into Data: Importing a Text, or, Clip, Chunk, and Split
Author:   R.C. Alvarado
```

**Purpose:**
* Demonstrate how to tokenize a raw text.
* Demonstrate how to map an OHCO onto the resulting dataframe of tokens.

# Set Up

In [69]:
import pandas as pd

In [70]:
data_home = "../data"

In [71]:
text_file = f"{data_home}/gutenberg/pg105.txt"
csv_file = f"{data_home}/output/austen-persuasion.csv" # The file we will create

In [72]:
OHCO = ['chap_num', 'para_num', 'sent_num', 'token_num']

# Import file into a dataframe

In [73]:
LINES = pd.DataFrame(open(text_file, 'r', encoding='utf-8-sig').readlines(), columns=['line_str'])
LINES.index.name = 'line_num'
LINES.line_str = LINES.line_str.str.replace(r'\n+', ' ', regex=True).str.strip()

In [74]:
# LINES.shape

In [75]:
LINES.sample(20)

,line_str
line_num,
4060,"returned again to Lyme, and without any presen..."
3052,"The conclusion of her visit, however, was dive..."
1895,suddenly resolving to walk to the end of the v...
3223,usefulness and ingenuity seemed to furnish him...
464,"had another excellent one at hand, for being e..."
4169,"every day; and in short, it seemed to have bee..."
1751,"would not like to leave him yourself, but you ..."
2296,should hear from him next; but as long as we c...
1312,she had expected rather more curiosity and sym...


# Extract Title 

In [76]:
title = LINES.loc[0].line_str.replace('The Project Gutenberg EBook of ', '')

In [77]:
print(title)

Persuasion, by Jane Austen


# Clip Cruft

In [78]:
clip_pats = [
    r"\*\*\*\s*START OF (?:THE|THIS) PROJECT", 
    r"\*\*\*\s*END OF (?:THE|THIS) PROJECT"
]

In [79]:
pat_a = LINES.line_str.str.match(clip_pats[0])
pat_b = LINES.line_str.str.match(clip_pats[1])

In [80]:
line_a = LINES.loc[pat_a].index[0] + 1
line_b = LINES.loc[pat_b].index[0] - 1

In [81]:
line_a, line_b

(19, 8372)

In [82]:
LINES = LINES.loc[line_a : line_b]

In [83]:
LINES.head(10)

,line_str
line_num,
19,
20,
21,
22,
23,Produced by Sharon Partridge and Martin Ward. ...
24,by Al Haines.
25,
26,
27,


In [84]:
LINES.tail(10)

,line_str
line_num,
8363,
8364,
8365,
8366,
8367,
8368,
8369,
8370,
8371,End of the Project Gutenberg EBook of Persuasi...


# Chunk by chapter

## Find all chapter headers

The regex will depend on the source text. You need to investigate the source text to figure this out.

In [85]:
chap_pat = r"^\s*(?:chapter|letter)\s+\d+"

In [86]:
chap_lines = LINES.line_str.str.match(chap_pat, case=False) # Returns a truth vector

In [87]:
LINES.loc[chap_lines] # Use as filter for dataframe

,line_str
line_num,
47,Chapter 1
306,Chapter 2
500,Chapter 3
786,Chapter 4
959,Chapter 5
1297,Chapter 6
1657,Chapter 7
1992,Chapter 8
2346,Chapter 9


## Assign numbers to chapters

In [88]:
LINES.loc[chap_lines, 'chap_num'] = [i+1 for i in range(LINES.loc[chap_lines].shape[0])]

In [89]:
LINES.loc[chap_lines]

,line_str,chap_num
line_num,,
47,Chapter 1,1.0
306,Chapter 2,2.0
500,Chapter 3,3.0
786,Chapter 4,4.0
959,Chapter 5,5.0
1297,Chapter 6,6.0
1657,Chapter 7,7.0
1992,Chapter 8,8.0
2346,Chapter 9,9.0


Notice that all lines that are not chapter headers have no chapter number assigned to them.

In [92]:
LINES.sample(10)

,line_str,chap_num
line_num,,
258,,NaN
3177,"and, compared with either of them, a little ma...",NaN
8371,End of the Project Gutenberg EBook of Persuasi...,NaN
5208,"judge properly and as a man of principle, this...",NaN
4604,,NaN
2695,"Their time and strength, and spirits, were, th...",NaN
519,gentlemen of the navy are well to deal with. ...,NaN
96,"these gifts, was the constant object of his wa...",NaN
5273,Anne could listen no longer; she could not eve...,NaN


## Forward-fill chapter numbers to following text lines

`ffill()` will replace null values with the previous non-null value.

In [93]:
LINES.chap_num = LINES.chap_num.ffill()

In [94]:
LINES.sample(10)

,line_str,chap_num
line_num,,
145,"To Lady Russell, indeed, she was a most dear a...",1.0
2641,"experience, that Captain Wentworth was not in ...",10.0
4978,Anne's astonishment increased. She could scar...,17.0
7319,"was to leave Bath at nine this morning, and do...",22.0
3179,conversation.,11.0
4567,"walked, that one handsome face would be follow...",15.0
2016,commonest civility required. Once so much to ...,8.0
339,They must retrench; that did not admit of a do...,2.0
34,,NaN


Notice that the lines taht precede our first chapter have no chapters, which is what we want. We need to decide whether to keep these lines as textual front matter or to dispose of them.

In [95]:
LINES.head(20)

,line_str,chap_num
line_num,,
19,,NaN
20,,NaN
21,,NaN
22,,NaN
23,Produced by Sharon Partridge and Martin Ward. ...,NaN
24,by Al Haines.,NaN
25,,NaN
26,,NaN
27,,NaN


## Clean up

In [96]:
LINES = LINES.dropna(subset=['chap_num']) # Remove everything before Chapter 1
# LINES = LINES.loc[~LINES.chap_num.isna()] # Remove everything before Chapter 1 (alternate method)
LINES = LINES.loc[~chap_lines] # Remove chapter heading lines; their work is done
LINES.chap_num = LINES.chap_num.astype('int') # Convert chap_num from float to int

In [97]:
LINES.sample(10)

,line_str,chap_num
line_num,,
6187,,20
3676,"and fear, from a return of her own insensibili...",12
3661,showed something to be wrong; and they had set...,12
6755,"hardly have much truth left.""",21
2784,Henrietta should just run down for a few minut...,10
288,relinquishing their comforts in a way not to b...,1
4598,ten o'clock. Could it be Mr Elliot? They kne...,15
848,"not a merely selfish caution, under which she ...",4
5933,no hurry to leave her; and presently with rene...,20


## Group lines into chapters

In [98]:
OHCO[:1]

['chap_num']

In [99]:
CHAPS = LINES.groupby(OHCO[:1]).line_str.apply(lambda x: '\n'.join(x)).to_frame('chap_str') # Make big string for each chapter

In [100]:
CHAPS.head(10)

,chap_str
chap_num,
1,"\n\nSir Walter Elliot, of Kellynch Hall, in So..."
2,"\n\nMr Shepherd, a civil, cautious lawyer, who..."
3,"\n\n""I must take leave to observe, Sir Walter,..."
4,"\n\nHe was not Mr Wentworth, the former curate..."
5,\n\nOn the morning appointed for Admiral and M...
6,\n\nAnne had not wanted this visit to Uppercro...
7,"\n\nA very few days more, and Captain Wentwort..."
8,\n\nFrom this time Captain Wentworth and Anne ...
9,\n\nCaptain Wentworth was come to Kellynch as ...


In [101]:
CHAPS['chap_str'] = CHAPS.chap_str.str.strip()

In [102]:
CHAPS.head(10)

,chap_str
chap_num,
1,"Sir Walter Elliot, of Kellynch Hall, in Somers..."
2,"Mr Shepherd, a civil, cautious lawyer, who, wh..."
3,"""I must take leave to observe, Sir Walter,"" sa..."
4,"He was not Mr Wentworth, the former curate of ..."
5,On the morning appointed for Admiral and Mrs C...
6,"Anne had not wanted this visit to Uppercross, ..."
7,"A very few days more, and Captain Wentworth wa..."
8,From this time Captain Wentworth and Anne Elli...
9,Captain Wentworth was come to Kellynch as to a...


So, now we have our text grouped by chapters.

# Split chapters into paragraphs 

We use Pandas' convenient `.split()` method with `expand=True`, followed by `.stack()`.
Note that this creates zero-based indexes.

In [103]:
para_pat = r'\n\n+'

In [104]:
# CHAPS['chap_str'].str.split(para_pat, expand=True).head()

In [105]:
PARAS = CHAPS['chap_str'].str.split(para_pat, expand=True).stack()\
    .to_frame('para_str').sort_index()
PARAS.index.names = OHCO[:2]

In [106]:
PARAS.head()

para_str
chap_num para_num                                                   
1        0         Sir Walter Elliot, of Kellynch Hall, in Somers...
         1                                 "ELLIOT OF KELLYNCH HALL.
         2         "Walter Elliot, born March 1, 1760, married, J...
         3         Precisely such had the paragraph originally st...
         4         Then followed the history and rise of the anci...

In [107]:
PARAS['para_str'] = PARAS['para_str'].str.replace(r'\n', ' ', regex=True)
PARAS['para_str'] = PARAS['para_str'].str.strip()
PARAS = PARAS[~PARAS['para_str'].str.match(r'^\s*$')] # Remove empty paragraphs

In [108]:
PARAS.head()

para_str
chap_num para_num                                                   
1        0         Sir Walter Elliot, of Kellynch Hall, in Somers...
         1                                 "ELLIOT OF KELLYNCH HALL.
         2         "Walter Elliot, born March 1, 1760, married, J...
         3         Precisely such had the paragraph originally st...
         4         Then followed the history and rise of the anci...

# Split paragraphs into sentences

In [109]:
# sent_pat = r'[.?!;:"]+'
sent_pat = r'[.?!;:]+'
SENTS = PARAS['para_str'].str.split(sent_pat, expand=True).stack()\
    .to_frame('sent_str')
SENTS.index.names = OHCO[:3]

In [110]:
SENTS = SENTS[~SENTS['sent_str'].str.match(r'^\s*$')] # Remove empty paragraphs
SENTS.sent_str = SENTS.sent_str.str.strip() # CRUCIAL TO REMOVE BLANK TOKENS

In [111]:
SENTS.head()

sent_str
chap_num para_num sent_num                                                   
1        0        0         Sir Walter Elliot, of Kellynch Hall, in Somers...
                  1         there he found occupation for an idle hour, an...
                  2         there his faculties were roused into admiratio...
                  3         there any unwelcome sensations, arising from d...
                  4         and there, if every other leaf were powerless,...

# Split sentences into tokens

In [112]:
token_pat = r"[\s',-]+"
TOKENS = SENTS['sent_str'].str.split(token_pat, expand=True).stack()\
    .to_frame('token_str')

In [113]:
TOKENS.index.names = OHCO[:4]

In [114]:
TOKENS

token_str
chap_num para_num sent_num token_num            
1        0        0        0                 Sir
                           1              Walter
                           2              Elliot
                           3                  of
                           4            Kellynch
...                                          ...
24       13       0        6                  of
                           7          Persuasion
                           8                  by
                           9                Jane
                           10             Austen

[85014 rows x 1 columns]

# Extract Vocabulary

In [115]:
TOKENS['term_str'] = TOKENS.token_str.replace(r'[\W_]+', '', regex=True).str.lower()
VOCAB = TOKENS.term_str.value_counts().to_frame('n').reset_index().rename(columns={'index':'term_str'})
VOCAB.index.name = 'term_id'

In [116]:
VOCAB

,term_str,n
term_id,,
0,the,3330
1,to,2808
2,and,2800
3,of,2572
4,a,1595
...,...,...
5755,slip,1
5756,faring,1
5757,glare,1


# Gathering by Content Object

In [117]:
def gather(ohco_level):
    global TOKENS
    level_name = OHCO[ohco_level-1].split('_')[0]
    df = TOKENS.groupby(OHCO[:ohco_level])\
        .token_str.apply(lambda x: x.str.cat(sep=' '))\
        .to_frame(f"{level_name}_str")
    return df

In [118]:
gather(1)

,chap_str
chap_num,
1,Sir Walter Elliot of Kellynch Hall in Somerset...
2,Mr Shepherd a civil cautious lawyer who whatev...
3,"""I must take leave to observe Sir Walter "" sai..."
4,He was not Mr Wentworth the former curate of M...
5,On the morning appointed for Admiral and Mrs C...
6,Anne had not wanted this visit to Uppercross t...
7,A very few days more and Captain Wentworth was...
8,From this time Captain Wentworth and Anne Elli...
9,Captain Wentworth was come to Kellynch as to a...


In [119]:
gather(2)

para_str
chap_num para_num                                                   
1        0         Sir Walter Elliot of Kellynch Hall in Somerset...
         1                                  "ELLIOT OF KELLYNCH HALL
         2         "Walter Elliot born March 1 1760 married July ...
         3         Precisely such had the paragraph originally st...
         4         Then followed the history and rise of the anci...
...                                                              ...
24       9         Anne satisfied at a very early period of Lady ...
         10        Her recent good offices by Anne had been enoug...
         11        Mrs Smith s enjoyments were not spoiled by thi...
         12                                                    Finis
         13        End of the Project Gutenberg EBook of Persuasi...

[1008 rows x 1 columns]

In [120]:
gather(3)

sent_str
chap_num para_num sent_num                                                   
1        0        0         Sir Walter Elliot of Kellynch Hall in Somerset...
                  1         there he found occupation for an idle hour and...
                  2         there his faculties were roused into admiratio...
                  3         there any unwelcome sensations arising from do...
                  4         and there if every other leaf were powerless h...
...                                                                       ...
24       11       4         Anne was tenderness itself and she had the ful...
                  5         His profession was all that could ever make he...
                  6         She gloried in being a sailor s wife but she m...
         12       0                                                     Finis
         13       0         End of the Project Gutenberg EBook of Persuasi...

[5612 rows x 1 columns]

# Save work to CSV

This is important -- will be used for homework.

In [121]:
TOKENS.to_csv(csv_file)

# Use Library

In [122]:
import sys
sys.path.append("../lib")
from textimporter import TextImporter

In [123]:
pg105 = TextImporter(src_file=text_file, ohco_pats=[('chap', chap_pat, 'm')], clip_pats=clip_pats)

In [124]:
pg105.OHCO

['chap_id', 'para_num', 'sent_num', 'token_num']

In [125]:
pg105.import_source()
pg105.parse_tokens()
pg105.extract_vocab()

Importing  ../data/gutenberg/pg105.txt
Clipping text
Parsing OHCO level 0 chap_id m by milestone ^\s*(?:chapter|letter)\s+\d+
line_str
Parsing OHCO level 1 para_num d by delimitter \n\n
Parsing OHCO level 2 sent_num d by delimitter [.?!;:]+
Parsing OHCO level 3 token_num d by delimitter [\s',-]+


In [126]:
pg105.TOKENS

token_str    term_str
chap_id para_num sent_num token_num                        
1       0        0        0                 Sir         sir
                          1              Walter      walter
                          2              Elliot      elliot
                          3                  of          of
                          4            Kellynch    kellynch
...                                         ...         ...
24      17       0        7                  of          of
                          8          Persuasion  persuasion
                          9                  by          by
                          10               Jane        jane
                          11             Austen      austen

[84904 rows x 2 columns]

In [127]:
pg105.VOCAB

,n,n_chars,p,s,i,h
term_str,,,,,,
the,3330,3,0.039221,25.496697,4.672238,0.183249
to,2808,2,0.033073,30.236467,4.918218,0.162658
and,2800,3,0.032978,30.322857,4.922334,0.162331
of,2572,2,0.030293,33.010886,5.044870,0.152824
a,1595,1,0.018786,53.231348,5.734204,0.107722
...,...,...,...,...,...,...
slip,1,4,0.000012,84904.000000,16.373545,0.000193
faring,1,6,0.000012,84904.000000,16.373545,0.000193
glare,1,5,0.000012,84904.000000,16.373545,0.000193
